# Imports:

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow import keras
from keras.layers import Embedding, Bidirectional, GRU, Dense, GlobalMaxPooling1D, SpatialDropout1D, Reshape
from keras.models import Sequential, load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
import numpy as np
from transformers import BertTokenizer, TFBertModel
from transformers import RobertaTokenizer, TFRobertaModel
from typing import List
import tensorflow as tf
from sklearn.model_selection import KFold
from node2vec import Node2Vec
from keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
import utils_harmful

In [2]:
import importlib
importlib.reload(utils_harmful)

<module 'utils_harmful' from '/Users/anaconstantinescu/Documents/Ana/an_4/licenta/utils_harmful.py'>

# Process Data:

In [37]:
# Extract tweets and labels 
df_en, y_labels = utils_harmful.process_dataset("./dataset/training/EXIST2023_training.json")

# Glove model train:

In [38]:
glove_model = KeyedVectors.load_word2vec_format('glove.6B/glove.6B.50d.txt', binary=False)
vector = glove_model['word']

# Trained model using Glove Embedding Layer, EarlyStpping and Cross Validation:

In [ ]:
max_sequence_length = 50

# Tokenize tweets
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_en['tweet']) # Fit tokenizer on tweets
sequences = tokenizer.texts_to_sequences(df_en['tweet'])  # Convert texts to sequences
word_index = tokenizer.word_index # Get word index

# Pad sequences
X_pad = pad_sequences(sequences, maxlen=max_sequence_length)

# Build the embedding matrix
embedding_matrix = np.zeros((len(word_index) + 1, 50))
for word, i in word_index.items():
    # Check if word is in GloVe embeddings
    if word in glove_model:
        embedding_matrix[i] = glove_model[word]

# Build the BiGRU model with Embedding layer
model = Sequential()
model.add(Embedding(len(word_index) + 1, 50, weights=[embedding_matrix], input_length=max_sequence_length, trainable=True))
model.add(Bidirectional(GRU(units=32, return_sequences=True)))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

early_stopping = keras.callbacks.EarlyStopping(monitor='loss', patience=3, baseline=None)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Iterate over each fold and evaluate
utils_harmful.kfold_train_evaluate(model, X_pad, y_labels, nr_splits=10, callback=early_stopping, epoch_num = 50)

In [25]:
max_sequence_length = 50

# Tokenize tweets
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_en['tweet']) # Fit tokenizer on tweets
sequences = tokenizer.texts_to_sequences(df_en['tweet'])  # Convert texts to sequences
word_index = tokenizer.word_index # Get word index

# Pad sequences
X_pad = pad_sequences(sequences, maxlen=max_sequence_length)

# Initialize the embedding matrix
embedding_matrix = np.zeros((len(X_pad), max_sequence_length, max_sequence_length))

# Build the embedding matrix as a 3d matrix, each tweet having a list an array of word embeddings for each word
for i, seq in enumerate(X_pad):
    for j, word_index in enumerate(seq):
            embedding_matrix[i, j, :] = glove_model[word_index]

# Build the BiGRU model
model = Sequential()
model.add(Bidirectional(GRU(units=32, return_sequences=True), input_shape=(max_sequence_length, 50)))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

early_stopping = keras.callbacks.EarlyStopping(monitor='loss', patience=3, baseline=None)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

utils_harmful.kfold_train_evaluate(model, embedding_matrix, y_labels, nr_splits=10, callback=early_stopping, epoch_num = 50)

Fold 1/10
Epoch 1/50


2024-02-26 19:29:34.774282: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


92/92 [==============================] - 4s 24ms/step - loss: 0.6756 - accuracy: 0.6228
Epoch 2/50
92/92 [==============================] - 2s 23ms/step - loss: 0.6299 - accuracy: 0.6545
Epoch 3/50
92/92 [==============================] - 2s 22ms/step - loss: 0.6009 - accuracy: 0.6780
Epoch 4/50
92/92 [==============================] - 2s 22ms/step - loss: 0.5826 - accuracy: 0.6999
Epoch 5/50
92/92 [==============================] - 2s 22ms/step - loss: 0.5648 - accuracy: 0.7183
Epoch 6/50
92/92 [==============================] - 2s 22ms/step - loss: 0.5534 - accuracy: 0.7207
Epoch 7/50
92/92 [==============================] - 2s 22ms/step - loss: 0.5345 - accuracy: 0.7282
Epoch 8/50
92/92 [==============================] - 2s 23ms/step - loss: 0.5201 - accuracy: 0.7398
Epoch 9/50
92/92 [==============================] - 2s 24ms/step - loss: 0.5041 - accuracy: 0.7531
Epoch 10/50
92/92 [==============================] - 2s 23ms/step - loss: 0.4910 - accuracy: 0.7664
Epoch 11/50
92/92 [=

Average Accuracy: 0.9669
Average Precision: 0.9525
Average Recall: 0.9558

# Hyperparameter Tuning: use Grid search:

In [ ]:
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

# Function to create the model
def create_model(units, optimizer):
    model = Sequential()
    model.add(Embedding(len(word_index) + 1, 50, weights=[embedding_matrix], input_length=max_sequence_length, trainable=True))
    model.add(Bidirectional(GRU(units=units, return_sequences=True), input_shape=(None, 50)))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Define hyperparameters to tune
param_grid = {
    'model__units': [16, 32],
    'batch_size': [16, 32],
    'model__optimizer': ['adam', 'rmsprop'],
}

# Initialize StratifiedKFold for cross-validation
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, min_delta=0.005)
grid = GridSearchCV(estimator=KerasClassifier(model=create_model, epochs=30),
                    param_grid=param_grid, scoring='accuracy', cv=5)

# Fit the GridSearchCV object to the data
grid_result = grid.fit(X_pad, y_labels, callbacks=[early_stopping])

# Print the best parameters
print("Best Parameters: ", grid_result.best_params_)
print("Best Accuracy: ", grid_result.best_score_)

# BERT:

In [16]:
# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased', trainable=False)

# Define maximum sequence length
max_sequence_length = 128

# Function to get BERT vectors for a dataset
def get_bert_vectors_for_dataset(dataset: List[dict]) -> tf.Tensor:
    bert_embeddings = []
    tweets = dataset["tweet"]  # Extract tweets from dataset
    
    for text in tweets:
        # Tokenize text using BERT tokenizer
        encoded_text = tokenizer(text, padding='max_length', truncation=True, max_length=max_sequence_length, return_tensors='tf')
        
        # Get BERT model outputs
        outputs = bert_model(encoded_text)
       
        # Extract last hidden states from BERT outputs
        last_hidden_states = outputs.last_hidden_state
        
        # Extract [CLS] token embedding (first token of each sequence)
        cls_embedding = last_hidden_states[:, 0, :]
        bert_embeddings.append(cls_embedding)

    # Concatenate BERT embeddings along the first axis
    bert_vectors = tf.concat(bert_embeddings, axis=0)
    # Add an additional dimension to BERT vectors
    bert_vectors_reshaped = tf.expand_dims(bert_vectors, axis=1) 

    return bert_vectors_reshaped


X_bert = get_bert_vectors_for_dataset(df_en)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

# RoBERTa:

In [26]:
# Initialize RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = TFRobertaModel.from_pretrained('roberta-base', trainable=False)

# Define maximum sequence length
max_sequence_length = 128

# Function to get RoBERTa vectors for a dataset
def get_roberta_vectors_for_dataset(dataset: List[dict], model, max_sequence_length) -> tf.Tensor:
    embeddings = []
    tweets = dataset["tweet"] # Extract tweets from dataset
    
    for text in tweets:
        # Tokenize text using RoBERTa tokenizer
        encoded_text = tokenizer(text, padding='max_length', truncation=True, max_length=max_sequence_length, return_tensors='tf')
        
        # Get RoBERTa model outputs
        outputs = model(encoded_text)

        # Extract last hidden states from RoBERTa outputs
        last_hidden_states = outputs.last_hidden_state

        # Calculate mean embedding over sequence length
        mean_embedding = tf.reduce_mean(last_hidden_states, axis=1)
    
        embeddings.append(mean_embedding)

    # Concatenate RoBERTa embeddings along the first axis
    vectors = tf.concat(embeddings, axis=0)
    # Add an additional dimension to RoBERTa vectors
    vectors_reshaped = tf.expand_dims(vectors, axis=1)

    return vectors_reshaped

X_roberta = get_roberta_vectors_for_dataset(df_en, roberta_model, max_sequence_length)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'roberta.embeddings.position_ids', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

# Trained model using BERT/RoBERTa Embedding Layer, EarlyStpping and Cross Validation:

In [24]:

def bert_roberta_evaluation(X: np.ndarray, y_labels: np.ndarray, n_splits: int, epochs: int) -> tuple:
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    model = Sequential()
    model.add(Bidirectional(GRU(units=32, return_sequences=True, input_shape=(None, 768), dtype='float32')))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(1, activation='sigmoid'))
    
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, baseline=None)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    utils_harmful.kfold_train_evaluate(model, X, y_labels, nr_splits=10, callback=early_stopping, epoch_num=epochs)

In [28]:
bert_roberta_evaluation(X_roberta, y_labels, n_splits=10, epochs=100)

Fold 1/10
Epoch 1/100
92/92 [==============================] - 6s 26ms/step - loss: 0.5977 - accuracy: 0.6746
Epoch 2/100
92/92 [==============================] - 1s 16ms/step - loss: 0.5203 - accuracy: 0.7285
Epoch 3/100
92/92 [==============================] - 1s 14ms/step - loss: 0.4854 - accuracy: 0.7613
Epoch 4/100
92/92 [==============================] - 1s 15ms/step - loss: 0.4618 - accuracy: 0.7793
Epoch 5/100
92/92 [==============================] - 2s 21ms/step - loss: 0.4520 - accuracy: 0.7834
Epoch 6/100
92/92 [==============================] - 1s 15ms/step - loss: 0.4385 - accuracy: 0.7872
Epoch 7/100
92/92 [==============================] - 1s 15ms/step - loss: 0.4316 - accuracy: 0.7920
Epoch 8/100
92/92 [==============================] - 2s 18ms/step - loss: 0.4282 - accuracy: 0.7943
Epoch 9/100
92/92 [==============================] - 2s 17ms/step - loss: 0.4216 - accuracy: 0.8025
Epoch 10/100
92/92 [==============================] - 2s 19ms/step - loss: 0.4178 - accura

Average Accuracy: 0.9650
Average Precision: 0.9490
Average Recall: 0.9514

In [ ]:
bert_roberta_evaluation(X_bert, y_labels, n_splits=10, epochs=100)

Average Accuracy: 0.9730
Average Precision: 0.9624
Average Recall: 0.9576


# Graph Embeddings:

## Create graph:

In [26]:
import networkx as nx

graph = nx.Graph()

# Add nodes for tweets
for tweet_id in df_en.index:
    graph.add_node(f"tweet_{tweet_id}")

# Add edges between tweets with mutual annotators
for annotator in df_en['annotators'].explode().unique():
    tweets_with_annotator = df_en[df_en['annotators'].apply(lambda x: annotator in x)].index
    for tweet1 in tweets_with_annotator:
        for tweet2 in tweets_with_annotator:
            if tweet1 != tweet2:
                common_annotators = len(set(df_en.at[tweet1, 'annotators']).intersection(df_en.at[tweet2, 'annotators']))
                cost = common_annotators  # Cost based on the number of common annotators
                graph.add_edge(f"tweet_{tweet1}", f"tweet_{tweet2}", cost=cost)

## Node2vec Embeddings:

In [27]:
# Define a custom weighted walk function
def weighted_walk(graph, node, prev_node, p, q):
    
    weights = []  # List to store weights for neighbors
    neighbors = []  # List to store neighboring nodes

    for neighbor, data in graph[node].items():
        # Consider the edge cost as the weight for each neighbor
        weight = data['cost'] if 'cost' in data else 1
        neighbors.append|(neighbor)
        if neighbor == prev_node:
            weights.append((weight / p))
        else:
            weights.extend(weight / q)
    
    weight_sum = sum(weights)

    # Calculate probabilities for each neighbor
    probabilities = [weight / weight_sum for weight in weights]

    # Choose next node based on probabilities
    next = np.random.choice(neighbors, size=1, p=probabilities)[0]

    return next

# Node2Vec Embedding layer
graph_embedding_size = 50  # You can adjust this size

node2vec = Node2Vec(graph, dimensions=graph_embedding_size, walk_length=30, num_walks=200, workers=4)
node2vec._walk = weighted_walk  # Override default walk function with custom one
node2vec_model = node2vec.fit(window=10, min_count=1)

# Generate Node2Vec embeddings for tweets
node2vec_embeddings = np.array([node2vec_model.wv[f"tweet_{tweet_id}"] for tweet_id in df_en.index])
node2vec_embeddings = pad_sequences(node2vec_embeddings, maxlen=graph_embedding_size, dtype="float32", value=0, truncating="post", padding="post")

# expand  embeddings structure for later use
node2vec_embeddings = tf.concat(node2vec_embeddings, axis=0)
node2vec_embeddings = tf.expand_dims(node2vec_embeddings, axis=1)


Computing transition probabilities:   0%|          | 0/3258 [00:00<?, ?it/s]

Generating walks (CPU: 4): 100%|██████████| 50/50 [00:58<00:00,  1.17s/it]


# Trained model using BERT/RoBERTa/Glove + node2vec Embedding Layer, EarlyStpping and Cross Validation:

## BERT/RoBERTa + node2vec

In [31]:
from keras.layers import Concatenate

def train_model(model_type, model_name, X):
    # Load model and tokenizer
    if model_type == 'BERT':
        model_pretrain = TFBertModel.from_pretrained(model_name, trainable=False)
    elif model_type == 'RoBERTa':
        model_pretrain = TFRobertaModel.from_pretrained(model_name, trainable=False)

    # Concatenate embeddings Bert/Robert with graph embeddings
    X_combined = Concatenate(axis=-1)([X, node2vec_embeddings])

    model = Sequential()
    model.add(Bidirectional(GRU(units=16, return_sequences=True), input_shape=(None, model_pretrain.config.hidden_size + graph_embedding_size)))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(1, activation='sigmoid'))

    callback = EarlyStopping(monitor='loss', patience=3, baseline=None)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train and evaluate model using k-fold cross-validation
    utils_harmful.kfold_train_evaluate(model, X_combined, y_labels, nr_splits=10, callback=early_stopping, epoch_num=100)

In [32]:
train_model('BERT', 'bert-base-uncased', X_bert)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Fold 1/10
Epoch 1/100
92/92 [==============================] - 7s 30ms/step - loss: 0.5943 - accuracy: 0.6859
Epoch 2/100
92/92 [==============================] - 1s 15ms/step - loss: 0.5155 - accuracy: 0.7432
Epoch 3/100
92/92 [==============================] - 2s 18ms/step - loss: 0.4695 - accuracy: 0.7694
Epoch 4/100
92/92 [==============================] - 1s 15ms/step - loss: 0.4550 - accuracy: 0.7783
Epoch 5/100
92/92 [==============================] - 2s 18ms/step - loss: 0.4333 - accuracy: 0.7923
Epoch 6/100
92/92 [==============================] - 2s 17ms/step - loss: 0.4392 - accuracy: 0.7892
Epoch 7/100
92/92 [==============================] - 1s 16ms/step - loss: 0.4175 - accuracy: 0.7998
Epoch 8/100
92/92 [==============================] - 1s 15ms/step - loss: 0.3930 - accuracy: 0.8172
Epoch 9/100
92/92 [==============================] - 1s 14ms/step - loss: 0.3836 - accuracy: 0.8261
Epoch 10/100
92/92 [==============================] - 1s 15ms/step - loss: 0.3642 - accura

Average Accuracy: 0.9767
Average Precision: 0.9652
Average Recall: 0.9681

In [33]:
train_model('RoBERTa', 'roberta-base', X_roberta)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'roberta.embeddings.position_ids', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

Fold 1/10
Epoch 1/100
92/92 [==============================] - 7s 33ms/step - loss: 0.6401 - accuracy: 0.6392
Epoch 2/100
92/92 [==============================] - 2s 17ms/step - loss: 0.5619 - accuracy: 0.6995
Epoch 3/100
92/92 [==============================] - 2s 17ms/step - loss: 0.4924 - accuracy: 0.7534
Epoch 4/100
92/92 [==============================] - 1s 15ms/step - loss: 0.4723 - accuracy: 0.7711
Epoch 5/100
92/92 [==============================] - 1s 15ms/step - loss: 0.4512 - accuracy: 0.7814
Epoch 6/100
92/92 [==============================] - 1s 16ms/step - loss: 0.4369 - accuracy: 0.7896
Epoch 7/100
92/92 [==============================] - 1s 14ms/step - loss: 0.4271 - accuracy: 0.7902
Epoch 8/100
92/92 [==============================] - 1s 14ms/step - loss: 0.4257 - accuracy: 0.8012
Epoch 9/100
92/92 [==============================] - 1s 16ms/step - loss: 0.4132 - accuracy: 0.7998
Epoch 10/100
92/92 [==============================] - 1s 14ms/step - loss: 0.4031 - accura

Average Accuracy: 0.9681
Average Precision: 0.9559
Average Recall: 0.9541


## GLOVE + node2vec:

In [32]:
# Combine GloVe and Node2Vec embeddings
combined_embeddings = np.concatenate((embedding_matrix, node2vec_embeddings), axis=1)

model = Sequential()
model.add(Bidirectional(GRU(units=32, return_sequences=True), input_shape=(max_sequence_length + 1, 50)))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3, baseline=None)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

utils_harmful.kfold_train_evaluate(model, combined_embeddings, y_labels, nr_splits=10, callback=early_stopping, epoch_num=100)

Fold 1/10
Epoch 1/100
92/92 [==============================] - 6s 25ms/step - loss: 0.6444 - accuracy: 0.6511
Epoch 2/100
92/92 [==============================] - 2s 23ms/step - loss: 0.6209 - accuracy: 0.6569
Epoch 3/100
92/92 [==============================] - 2s 23ms/step - loss: 0.5992 - accuracy: 0.6729
Epoch 4/100
92/92 [==============================] - 2s 23ms/step - loss: 0.5803 - accuracy: 0.7046
Epoch 5/100
92/92 [==============================] - 2s 23ms/step - loss: 0.5647 - accuracy: 0.7135
Epoch 6/100
92/92 [==============================] - 2s 25ms/step - loss: 0.5442 - accuracy: 0.7282
Epoch 7/100
92/92 [==============================] - 2s 24ms/step - loss: 0.5306 - accuracy: 0.7408
Epoch 8/100
92/92 [==============================] - 2s 22ms/step - loss: 0.5106 - accuracy: 0.7551
Epoch 9/100
92/92 [==============================] - 2s 22ms/step - loss: 0.4945 - accuracy: 0.7647
Epoch 10/100
92/92 [==============================] - 2s 22ms/step - loss: 0.4786 - accura

Average Accuracy: 0.9660
Average Precision: 0.9534
Average Recall: 0.9381